# Setup

In [1]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid

from agent import create_agent_role, create_lambda_role
from agent import create_dynamodb, create_lambda, invoke_agent_helper

## Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-east-1', '161242380044')

# Setting up Agent's information

In [2]:
suffix = f"{region}-{account_id}"
agent_name = 'lol-coach-agent'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of player data"
agent_instruction = """
You are an expert League of Legends coach agent, helping players retrieve game data and create a new League of Legends player profile.
"""

agent_action_group_description = """
Actions for retrieving game data of a specific player and creating a new League of Legends player profile.
"""

agent_action_group_name = "DataRetrievalActionGroup"

# Select Foundation Model

In [3]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        ('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
        ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
print(agent_foundation_model_selector)

agent_foundation_model = agent_foundation_model_selector.value
print(agent_foundation_model)

Dropdown(description='FM:', options=(('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'), ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')), value='anthropic.claude-3-sonnet-20240229-v1:0')
anthropic.claude-3-sonnet-20240229-v1:0


# Creating DynamoDB Table

In [4]:
table_name = 'player_data'
create_dynamodb(table_name)

Creating table player_data...
Table player_data created successfully!


# Creating Lambda Function

In [5]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('player_data')

def get_named_parameter(event, name):
    """
    Get a parameter from the lambda event
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']

def get_player_data(player_id):
    """
    Retrieve game data of a specific player
    
    Args:
        player_id (string): The unique identifier for the player.
    """
    try:
        response = table.get_item(Key={'player_id': player_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No data found with ID {player_id}'}
    except Exception as e:
        return {'error': str(e)}
    
def create_lol_player(player_name, rank, region, main_role, favorite_champion):
    """
    Create a new League of Legends player profile.
    
    Args:
        player_name (string): The in-game name of the player.
        rank (string): The current rank of the player (e.g. 'Gold IV', 'Diamond I').
        region (string): The server region (e.g. 'NA', 'EUW', 'SG').
        main_role (string): The player’s main role (e.g. 'Top', 'Jungle', 'Mid', 'ADC', 'Support').
        favorite_champion (string): The champion the player uses most frequently.
    """
    try:
        player_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'player_id': player_id,
                'player_name': player_name,
                'rank': rank,
                'region': region,
                'main_role': main_role,
                'favorite_champion': favorite_champion
            }
        )
        return {'player_id': player_id}
    except Exception as e:
        return {'error': str(e)}
    
def lambda_handler(event, context):
    # get the action group used during the invocation of the lambda function
    actionGroup = event.get('actionGroup', '')
    
    # name of the function that should be invoked
    function = event.get('function', '')
    
    # parameters to invoke function with
    parameters = event.get('parameters', [])

    if function == 'get_player_data':
        player_id = get_named_parameter(event, "player_id")
        if player_id:
            response = str(get_player_data(player_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing player_id parameter'}}
    
    elif function == 'create_lol_player':
        player_name = get_named_parameter(event, "player_name")
        rank = get_named_parameter(event, "rank")
        region = get_named_parameter(event, "region")
        main_role = get_named_parameter(event, "main_role")
        favorite_champion = get_named_parameter(event, "favorite_champion")
        
        if all([player_name, rank, region, main_role, favorite_champion]):
            response = str(create_lol_player(player_name, rank, region, main_role, favorite_champion))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Overwriting lambda_function.py


In [6]:
lambda_iam_role = create_lambda_role(agent_name, table_name)
lambda_function_name = f'{agent_name}-lambda'
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

# Creating Agent

In [7]:
agent_role = create_agent_role(agent_name, agent_foundation_model)
print(agent_role)

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
print(response)

agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

{'Role': {'Path': '/', 'RoleName': 'AmazonBedrockExecutionRoleForAgents_lol-coach-agent', 'RoleId': 'AROASLCWLO4GENFDXKKU4', 'Arn': 'arn:aws:iam::161242380044:role/AmazonBedrockExecutionRoleForAgents_lol-coach-agent', 'CreateDate': datetime.datetime(2025, 10, 11, 7, 32, 46, tzinfo=tzutc()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'bedrock.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}, 'MaxSessionDuration': 3600, 'RoleLastUsed': {'LastUsedDate': datetime.datetime(2025, 10, 11, 13, 36, 23, tzinfo=tzutc()), 'Region': 'us-east-1'}}, 'ResponseMetadata': {'RequestId': '8da55512-9404-4bd1-bdbd-5f8ef42231a2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 13:55:51 GMT', 'x-amzn-requestid': '8da55512-9404-4bd1-bdbd-5f8ef42231a2', 'content-type': 'text/xml', 'content-length': '1006'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '43c39ffd-62ad-46f9-b667-e36c4aedb763', 'HTTPStatusCode': 202, 'HT

# Create Agent Action Group

In [8]:
agent_functions = [
    {
        'name': 'get_player_data',
        'description': 'Retrieve game data of a specific player',
        'parameters': {
            "player_id": {
                "description": "The unique identifier for the player",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_lol_player',
        'description': 'Create a new League of Legends player profile',
        'parameters': {
            "player_name": {
                "description": "The in-game name of the player",
                "required": True,
                "type": "string"
            },
            "rank": {
                "description": "The current rank of the player (e.g. 'Gold IV', 'Diamond I')",
                "required": True,
                "type": "string"
            },
            "region": {
                "description": "The server region (e.g. 'NA', 'EUW', 'SG')",
                "required": True,
                "type": "string"
            },
            "main_role": {
                "description": "The player’s main role (e.g. 'Top', 'Jungle', 'Mid', 'ADC', 'Support')",
                "required": True,
                "type": "string"
            },
            "favorite_champion": {
                "description": "The champion the player uses most frequently",
                "required": True,
                "type": "string"
            }
        }
    }
]

# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

print(agent_action_group_response)

{'ResponseMetadata': {'RequestId': '65033011-b6e7-48eb-a1ac-1afcd9891bb0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 13:56:23 GMT', 'content-type': 'application/json', 'content-length': '1460', 'connection': 'keep-alive', 'x-amzn-requestid': '65033011-b6e7-48eb-a1ac-1afcd9891bb0', 'x-amz-apigw-id': 'SSRBHH2KoAMEWZQ=', 'x-amzn-trace-id': 'Root=1-68ea6206-041b0252702768d125569a52'}, 'RetryAttempts': 0}, 'agentActionGroup': {'agentId': 'OY84EPHPDS', 'agentVersion': 'DRAFT', 'actionGroupId': 'MAGRSZPJ3U', 'actionGroupName': 'DataRetrievalActionGroup', 'description': '\nActions for retrieving game data of a specific player and creating a new League of Legends player profile.\n', 'createdAt': datetime.datetime(2025, 10, 11, 13, 56, 22, 929705, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2025, 10, 11, 13, 56, 22, 929705, tzinfo=tzutc()), 'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:161242380044:function:lol-coach-agent-lambda'}, 'functionSchema': {'fu

## Allowing bedrock to invoke lambda function

In [9]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': '801a5b23-aa11-4080-9a4b-d88e6d6fa53c', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 13:56:23 GMT', 'content-type': 'application/json', 'content-length': '361', 'connection': 'keep-alive', 'x-amzn-requestid': '801a5b23-aa11-4080-9a4b-d88e6d6fa53c'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"allow_bedrock_OY84EPHPDS","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:161242380044:function:lol-coach-agent-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:161242380044:agent/OY84EPHPDS"}}}'}


## Preparing agent

In [10]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': '81df7c21-4cff-4b13-944a-868b906669db', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 13:56:24 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '81df7c21-4cff-4b13-944a-868b906669db', 'x-amz-apigw-id': 'SSRBZERZoAMEZSw=', 'x-amzn-trace-id': 'Root=1-68ea6208-20308aa8279642c15911a0b4'}, 'RetryAttempts': 0}, 'agentId': 'OY84EPHPDS', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 10, 11, 13, 56, 24, 832545, tzinfo=tzutc())}


# Invoking Agent

In [11]:
alias_id = 'TSTALIASID'

In [12]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to create a new League of Legends player profile for a user named “ShadowFox”, ranked Diamond I, playing in the Singapore server, main role Mid, and favorite champion Yasuo."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The new League of Legends player profile has been successfully created with the following details:

Player Name: ShadowFox
Rank: Diamond I  
Server Region: SG (Singapore)
Main Role: Mid
Favorite Champion: Yasuo
Player ID: cb0a7d83
CPU times: user 32.5 ms, sys: 4.29 ms, total: 36.8 ms
Wall time: 10.1 s


In [15]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to retrieve the favorite champion for the player with the id '5130f179'."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The favorite champion for the player with id '5130f179' is Yasuo.
CPU times: user 27.3 ms, sys: 4.37 ms, total: 31.7 ms
Wall time: 5.77 s
